In [1]:
!apt-get install
!pip install torchattacks

zsh:1: command not found: apt-get
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.5.0
    Uninstalling urllib3-2.5.0:
      Successfully uninstalled urllib3-2.5.0
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11
  Attempting uninstall: requests90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/5 [idna]
    Found existing installation: requests 2.32.590m━━━━━━━━━━━━━━━ 3/5 [requests]
    Uninstalling requests-2.32.5:╺━━━━━━━━━━━━━━━ 3/5 [requests]
      Successfully uninstalled requests-2.32.5m━━━━━━━━━━━━━━━ 3/5 [requests]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [torchattacks][requests]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab-server 2.27.3 requires requests>=2.31, but you have requests 2.25.1 which is incompatible.


In [2]:
!pip install wandb

In [8]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import torchvision
import matplotlib.pyplot as plt
import numpy as np
import random
import torchattacks
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA


In [4]:
x = np.array([[random.randint(0, 1) for _ in range(100)] for _ in range(100)])
x

array([[0, 1, 0, ..., 1, 0, 1],
       [0, 1, 0, ..., 0, 1, 1],
       [0, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 1, 0],
       [1, 0, 1, ..., 1, 1, 1]])

In [5]:
y = []
for i in (range(x.shape[0])):
  if(x[i].sum() % 2 != 0): # Check if the sum is odd
    y.append(1)
  else: # If not odd, it's even
    y.append(0)
y = np.array(y)

In [6]:
y

array([1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0,
       1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1])

In [10]:
dimensions = [90, 80, 70, 60, 50, 40, 30, 20, 10, 5]

for n_components in dimensions:
    print(f"\n--- Running with PCA dimension: {n_components} ---")

    # Apply PCA for dimensionality reduction
    pca = PCA(n_components=n_components)
    x_pca = pca.fit_transform(x)

    # Perform KMeans clustering on the PCA-transformed data
    kmeans = KMeans(n_clusters=2, random_state=0, n_init=10)
    clusters = kmeans.fit_predict(x_pca)

    # Count the number of points in each cluster
    cluster_sizes = np.bincount(clusters)
    print("Cluster sizes:")
    for i, size in enumerate(cluster_sizes):
        print(f"  Cluster {i}: {size} points")

    # Calculate the sum of y labels for each cluster and divide by cluster size
    cluster_losses = {}
    for cluster_label in np.unique(clusters):
        # Get the indices of points in the current cluster
        indices = np.where(clusters == cluster_label)[0]

        # Get the corresponding y values
        y_in_cluster = y[indices]

        # Calculate the sum of y values and the cluster size
        sum_y = np.sum(y_in_cluster)
        cluster_size = len(y_in_cluster)

        # Calculate the loss (sum of y / cluster size)
        loss = sum_y / cluster_size if cluster_size > 0 else 0
        cluster_losses[cluster_label] = loss

    # Print the loss for each cluster
    print("Cluster losses:")
    for cluster_label, loss in cluster_losses.items():
        print(f"  Cluster {cluster_label}: Loss = {loss:.4f}")


--- Running with PCA dimension: 90 ---
Cluster sizes:
  Cluster 0: 60 points
  Cluster 1: 40 points
Cluster losses:
  Cluster 0: Loss = 0.4667
  Cluster 1: Loss = 0.5000

--- Running with PCA dimension: 80 ---
Cluster sizes:
  Cluster 0: 51 points
  Cluster 1: 49 points
Cluster losses:
  Cluster 0: Loss = 0.4510
  Cluster 1: Loss = 0.5102

--- Running with PCA dimension: 70 ---
Cluster sizes:
  Cluster 0: 59 points
  Cluster 1: 41 points
Cluster losses:
  Cluster 0: Loss = 0.5085
  Cluster 1: Loss = 0.4390

--- Running with PCA dimension: 60 ---
Cluster sizes:
  Cluster 0: 49 points
  Cluster 1: 51 points
Cluster losses:
  Cluster 0: Loss = 0.5510
  Cluster 1: Loss = 0.4118

--- Running with PCA dimension: 50 ---
Cluster sizes:
  Cluster 0: 54 points
  Cluster 1: 46 points
Cluster losses:
  Cluster 0: Loss = 0.5185
  Cluster 1: Loss = 0.4348

--- Running with PCA dimension: 40 ---
Cluster sizes:
  Cluster 0: 53 points
  Cluster 1: 47 points
Cluster losses:
  Cluster 0: Loss = 0.5094
 

In [11]:
# Calculate the sum of y labels for each cluster and divide by cluster size
cluster_losses = {}
for cluster_label in np.unique(clusters):
    # Get the indices of points in the current cluster
    indices = np.where(clusters == cluster_label)[0]

    # Get the corresponding y values
    y_in_cluster = y[indices]

    # Calculate the sum of y values and the cluster size
    sum_y = np.sum(y_in_cluster)
    cluster_size = len(y_in_cluster)

    # Calculate the loss (sum of y / cluster size)
    loss = sum_y / cluster_size if cluster_size > 0 else 0
    cluster_losses[cluster_label] = loss

# Print the loss for each cluster
for cluster_label, loss in cluster_losses.items():
    print(f"Cluster {cluster_label}: Loss = {loss:.4f}")

Cluster 0: Loss = 0.4524
Cluster 1: Loss = 0.5000


In [14]:
(100+93+75)/3 * 0.25 + 95 *0.25 + (89+99)/2 * 0.5

93.08333333333333